sklearn의 model_selection
- 학습 데이터와 테스트 데이터 셋 분리  ---> train_teat_split() 사용 [ 홀드아웃 방식 ]
- 교차 검증 분할 및 평가
- Estimator의 하이퍼 파라미터 튜닝

실습 - 복습

잘못된 모델

In [5]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris_data = load_iris()
dt_clf = DecisionTreeClassifier()

train_data = iris_data.data
train_label = iris_data.target

# 학습 진행
dt_clf.fit(train_data, train_label)

# 테스트
pred = dt_clf.predict(train_data)
print("예측 정확도 : ", accuracy_score(train_label, pred))

예측 정확도 :  1.0


제대로 예측 - 학습/테스트 데이터 셋 분리 및 예측

In [30]:
from sklearn.model_selection import train_test_split

# 학습/테스트 분할
X_train, X_test,  y_train, y_test =\
train_test_split(iris_data.data, iris_data.target, test_size = 0.3,
                random_state= 3)

In [31]:
# 학습 진행
dt_clf.fit(X_train, y_train)

# 테스트
pred = dt_clf.predict(X_test)
print("예측 정확도 : ", accuracy_score(y_test, pred))

예측 정확도 :  0.9777777777777777


pandas 이용 데이터셋 분할

In [33]:
import pandas as pd

In [38]:
def label_name(label):
    if label == 0:
        name = iris.target_names[0]
    elif label == 1:
        name = iris.target_names[1]
    else:
        name = iris.target_names[2]
    return name

In [37]:
iris_df = pd.DataFrame(iris_data.data, columns = iris_data.feature_names)
iris_df['target'] = iris_data.target

# 라벨 명 넣기
# iris_df['label_name'] = \
# iris_df.apply(lambda x : label_name(x['label']), axis = 1)

iris_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [39]:
feature_df = iris_df.iloc[:,:-1]
target_df = iris_df.iloc[:,-1]

# 학습/테스트 분할
X_train, X_test,  y_train, y_test =\
train_test_split(feature_df, target_df, test_size = 0.3,
                random_state= 4)

In [40]:
type(X_train)

pandas.core.frame.DataFrame

In [44]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도 : {0:0.3f}'.format(accuracy_score(y_test, pred)))

예측 정확도 : 0.978


### 교차 검증(Cross Validation, CV)
- k-fold Cross Vaildation 이라고도 함
- 전체 데이터 세트를 임의로 k개의 그룹으로 나누고, 그 가운데 하나의 그룸을 돌아가면서 테스트, 데이터 세트로, 나머지 k-1개 그룹은 학습용 데이터 셋으로 사용하는 방법
- 별도의 여러 세트로 구성된 학습 데이터 셋과 검증 데이터 셋에서 학습과 평가를 수행

- 사용 목적
    - 데이터에 적합한 알고리즘인지 판별
    - 모델에 적절한 hyperparameter를 찾기 위해
    - 과대적합 예방
    - 데이터 편중을 막기 위해

**K폴드 교차 검증 프로세스 구현**
1. KFold 클래스 = 폴드 세트로 분리
    - kfold = KFold(n_splits = 5)
2. split() : 폴드 데이터 셋으로 분리
    - kfold.split(features)
    - 각 폴드마다
        - 학습용, 검증용, 테스트 데이터 추출
        - 학습용 및 예측 수행
        - 정확도 측정
3. 최종 평균 정확도 계산

In [49]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
feature = iris.data
label = iris.target

feature.shape

(150, 4)

In [50]:
dt_clf = DecisionTreeClassifier(random_state = 156)

# 5개의 폴드 세트로 분리
kfold = KFold(n_splits = 5)

# 폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuracy = []


In [51]:
kfold.split(feature)

<generator object _BaseKFold.split at 0x000001B217AB2BC8>

In [52]:
for train_index, test_index in kfold.split(feature):
    print(train_index, test_index)

[ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 1

In [53]:
# 테스트 진행
for train_index, test_index in kfold.split(feature):
    X_train, X_test = feature[train_index], feature[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    
    acc = np.round(accuracy_score(y_test,pred), 3)
    
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print(f'정확도 : {acc}, 학습 데이터 크기 : {train_size}, 검증 데이터 크기 : {test_size}')
    cv_accuracy.append(acc)

print(f'평균 검증 정확도 : {np.mean(cv_accuracy)}')

정확도 : 1.0, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
정확도 : 0.967, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
정확도 : 0.867, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
정확도 : 0.933, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
정확도 : 0.733, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
평균 검증 정확도 : 0.9
